# Poland

In [30]:
import pandas as pd
import geopandas as gpd
from bs4 import BeautifulSoup
import numpy as np
import requests

pd.options.mode.chained_assignment = None

In [31]:
geo = gpd.read_file('../../data/shapefiles/pol/gadm36_POL_3.shp')

In [32]:
df = pd.read_excel('../../data/raw/pol.xls', dtype={'TERYT': str})

In [33]:
df = df[list(df.columns[:6]) + ['Kart wyjęte z urn']]

In [34]:
df.columns = ['okr', 'id', 'municipality', 'county', 'obw', 'eligible', 'voters']

In [35]:
# Should be 23.83
df.voters.sum() / df.eligible.sum()

0.2382143908758356

In [36]:
df['turnout'] = df.voters / df.eligible * 100

In [37]:
warszawa = df[df.county.str.contains('Warszawa')]
warszawa_eligible = warszawa.eligible.sum()
warszawa_voters = warszawa.voters.sum()
warszawa = warszawa.iloc[0]
warszawa.eligible = warszawa_eligible
warszawa.voters = warszawa_voters
warszawa.municipality = 'Warszawa'
warszawa.id = '146501'
warszawa.turnout = warszawa.voters / warszawa.eligible * 100

In [38]:
df = df[df.county != 'Warszawa']
df = df.append(warszawa)

In [39]:
geo['id'] = geo.CC_3.str[:-1]
geo = geo.merge(df, on='id', how='left')

In [40]:
geo = geo[['NAME_3', 'turnout', 'geometry']]
geo = geo.rename(columns={'NAME_3': 'name'})
geo['country'] = 'pol'

In [44]:
geo.to_file('../../data/processed/pol')